In [79]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/10/08 13:58
# @Author  : Wang Yujia
# @File    : PT_demo.ipynb
# @Description : 1. 主要是复现一下table 1的结果 2. SA的内容在SA_for_PT_model.ipynb上

# 1. Preparations
## 1.1 全局设置

In [53]:
# auction setting
b = 2
V = 6
d = 1

T = 3

import numpy as np
import pandas as pd
import sympy
#from sko.SA import SA
from scipy import optimize
from tqdm.notebook import tqdm

# 2. 验证Table.1
## 2.1 With EUT

In [ ]:
print("With EUT\n")
#x = sympy.Symbol('x')
def func(x):
    return (x[0]*(-b)+(1-x[0])*(V-t*d-b))
p=[]
for t in range(1,T+1):
    p.append(float((sympy.nsolve(func,x,(0,1)))))
    print("p[{0}]: {1}".format(t,p[t-1]))

## 2.2 With PT
1. 为了验证Table 1但是failed，只有最后一组[1,1,2.25]的第一个结果对不上
2. 侧面说明`C=[0,0,0,b]`是正确的设置？

In [104]:
# prob. weighting function
def omg(p):
    tmp = p**delta
    return tmp*((tmp + (1-p)**delta)**(-1/delta))

# value function
def f_x(x):
    return x**alpha

# for PT
alpha = 1
delta = 1
labda = 2.25

print("With PT\n")
x = sympy.Symbol('x')
p_2=[]
C=[0,0,0,b]
for t in range(1,T+1):
        #C = (t-1)*b*0.2
        tmp = V-d*(t)-C[t]-b
        if(tmp > 0):
            print(">0")
            func_1 = (labda*f_x(C[t]) + omg(x)*(-labda)*f_x(C[t]+b) + omg(1-x)*f_x(tmp))
            p_2.append((sympy.nsolve(func_1,(0,1), solver='bisect', verify=False)))
        else:
            print("<=0")
            func_2 = (-f_x(C[t]) + omg(x)*(f_x(C[t]+b))+(1-omg(x))*f_x(-tmp))
            p_2.append((sympy.nsolve(func_2, (0,1), solver='bisect', verify=False)))
        print("p[{0}]: {1}".format(t,p_2[t-1]))

With PT

>0
p[1]: 0.400000000000000
>0
p[2]: 0.307692307692308
<=0
p[3]: 0.333333333333333


# 3. PA test
1. 目的是求一个uniq auction下的u，观察过程与debug
## 3.1 preparations
1. v b d max_T都来自真实值

In [92]:
# auction setting
b = 0.75
v = 169.99
d = 0.15

# for PT
alpha = 0.88
delta = 0.65
labda = 2.25

# max duration
max_T = 831

## 3.2 function definition
1. 根据Eq(7)-(9)

In [106]:
def C(t,b):
    return 0.2*t*b

def OMEGA(p,delta):
    tmp = p**delta
    return tmp*((tmp + (1-p)**delta)**(-1/delta))

def valuation(x,alpha):
    return (1-sympy.E**(-alpha*np.abs(x)))/alpha
    # when x < 0, in fact, it shoule be : (-labda)*(1-sympy.E**(alpha*x))/alpha

def f_Equi(t,v,d,b,alpha,labda,delta):
    u = sympy.Symbol('u')

    tmp = v-d*t-C(t-1,b) - b

    func_1 = (labda*valuation(C(t-1,b),alpha) - labda*OMEGA(u,delta)*valuation((C(t-1,b)+b),alpha) + OMEGA(1-u,delta)*valuation(tmp,alpha))
    func_2 = (-valuation(C(t-1,b),alpha) + OMEGA(u,delta)*valuation((C(t-1,b)+b),alpha) + (1-OMEGA(u,delta))*valuation(tmp,alpha))

    if(tmp >= 0):
        return sympy.nsolve(func_1,(0,1),solver='bisect', verify=False)
    else:
        return sympy.nsolve(func_2,(0,1),solver='bisect', verify=False)

## 3.3 implement

In [107]:
# solve for u from Equi. condt.
U_tmp = [0]*(max_T+1)
U_tmp[0] = 1
for t in tqdm(range(1,max_T+1),desc="solve for u"):
    U_tmp[t] = f_Equi(t,v,d,b,alpha,labda,delta)

solve for u:   0%|          | 0/831 [00:00<?, ?it/s]

KeyError: 'pop from an empty set'